In [1]:
! pip install psycopg2

In [2]:
! python3 -m pip install requests

In [3]:
# import postgres
import psycopg2
import requests

In [4]:
# initiate connection

conn = psycopg2.connect(
    host = 'localhost'
    , user = 'ayo' # your username
    , password = 'password' # your password
    , database = 'bissan' # your database name
)

In [5]:
# create a cursor
cursor = conn.cursor()

## Question 1:
How many employees spent above their given stipend? Extract the name of the employee, the assigned stipend, and the actual amount spent.

In [10]:
import pandas as pd

In [25]:
# Extract employees and stipends data from db and compare

query = """
SELECT DISTINCT name, d.entitled_bonus as stipend 
FROM employees e
LEFT JOIN departments d
ON e.department_id = d._id
"""

cursor.execute(query)

In [26]:
all_employees_stipend = cursor.fetchall()

In [27]:
cursor.description #This will list the columns we are using i.e header of the extracted data from db

(Column(name='name', type_code=1043), Column(name='stipend', type_code=23))

In [32]:
employees_df = pd.DataFrame(all_employees_stipend, columns=['name', 'stipend'])

In [33]:
employees_df

,name,stipend
0,Robert Green,6832
1,James Sawyer,6832
2,Dieter Klinger,7377
3,Chase Fabrizio,6913
4,Scott Davis,6913
...,...,...
408,Lisa Killen,6086
409,Jason Giza,6832
410,Dana Stroman,7377
411,Joseph Webster,6832


In [35]:
employees_df[employees_df['name'].duplicated()]

,name,stipend
14,John Bentley,6086
133,Thomas Jordan,6086
146,Sean Renauer,6832
155,Mitsuko Herrera,6913
164,Catherine Brewer,6086
175,Sean Renauer,6913
320,Russ Hamill,6913
333,Dimitry Ruvin,6913
343,Darren Crandell,6832
375,Mark Sheelor,7377


In [36]:
#To get the actual amount spent, the info is to be extracted from the travel dataset saved in dir as cleaned_bissan_travels_data.csv

travels_dataset = pd.read_csv('cleaned_bissan_travels_data.csv') # read the csv

In [37]:
travels_dataset #The below shows that pandas indexed the data so we have double indexing, remove the index tagged as Unnamed: 0

,Unnamed: 0,Employee,Travel Start Date,Travel End Date,Actual Total Expenses,Purpose Of Travel,State,City
0,0,Aaron Salter,2017-05-31,2017-06-02,644.10,Conference,MD,Ocean City
1,1,Abigail Ratnofsky,2016-01-24,2016-01-29,2962.70,Conference,CA,San Diego
2,2,Adam Kisthardt,2018-10-04,2018-10-09,1891.19,Conference,FL,Orlando
3,3,Adam P. Jones,2016-06-19,2016-06-21,324.36,Conference,MD,Ocean City
4,4,Alan Butsch,2019-11-19,2019-11-22,2141.51,Conference,CA,San Diego
...,...,...,...,...,...,...,...,...
413,413,Warren Jensen,2017-11-06,2017-11-09,502.45,Conference,MD,Ocean City
414,414,Warp Jensen,2019-03-25,2019-03-26,78.00,Conference,MD,Ocean City
415,415,Whitney Kujawa,2017-11-06,2017-11-09,567.00,Conference,MD,Ocean City
416,416,William Kinna,2017-05-21,2017-05-26,2163.67,Conference,NC,Charlotte


In [38]:
#Using the .drop function from pandas, we define which column to drop and axis set 1 for column
travels_dataset = travels_dataset.drop('Unnamed: 0', axis=1)

In [193]:
travels_dataset #To view dataframe 

,Employee,Travel Start Date,Travel End Date,Actual Total Expenses,Purpose Of Travel,State,City
0,Aaron Salter,2017-05-31,2017-06-02,644.10,Conference,MD,Ocean City
1,Abigail Ratnofsky,2016-01-24,2016-01-29,2962.70,Conference,CA,San Diego
2,Adam Kisthardt,2018-10-04,2018-10-09,1891.19,Conference,FL,Orlando
3,Adam P. Jones,2016-06-19,2016-06-21,324.36,Conference,MD,Ocean City
4,Alan Butsch,2019-11-19,2019-11-22,2141.51,Conference,CA,San Diego
...,...,...,...,...,...,...,...
413,Warren Jensen,2017-11-06,2017-11-09,502.45,Conference,MD,Ocean City
414,Warp Jensen,2019-03-25,2019-03-26,78.00,Conference,MD,Ocean City
415,Whitney Kujawa,2017-11-06,2017-11-09,567.00,Conference,MD,Ocean City
416,William Kinna,2017-05-21,2017-05-26,2163.67,Conference,NC,Charlotte


In [44]:
employees_expense = travels_dataset[['Employee', 'Actual Total Expenses']] # To retrieve Employee and Actual Total Expenses

In [47]:
employees_expense[employees_expense['Employee'].duplicated()]

,Employee,Actual Total Expenses
26,Barbara Natoli,991.12
34,Bill Ferretti,353.22
52,Catherine Brewer,4925.24
80,Darren Crandell,1995.22
99,Dimitry Ruvin,2195.32
189,John Bentley,1119.22
229,Kevin Parker,1899.98
255,Lissa Bales,258.64
259,Mackell Heflin,2964.20
274,Mark Sheelor,1411.34


In [55]:
#This will get the total expenses for each employee
employees_expense = employees_expense.groupby(by='Employee').sum()
employees_expense = employees_expense.reset_index() #This will reset the index to pandas' default

In [56]:

employees_df = employees_df.drop_duplicates(subset='name') #Drop duplicated names

In [58]:
result = employees_expense.merge(employees_df, how='inner', left_on='Employee', right_on='name')

In [59]:
result.head(10)

,Employee,Actual Total Expenses,name,stipend
0,Aaron Salter,644.10,Aaron Salter,7377
1,Abigail Ratnofsky,2962.70,Abigail Ratnofsky,6086
2,Adam Kisthardt,1891.19,Adam Kisthardt,7377
3,Adam P. Jones,324.36,Adam P. Jones,6832
4,Alan Butsch,2141.51,Alan Butsch,6913
5,Alberto Hook,1856.49,Alberto Hook,6913
6,Alberto Rein,330.08,Alberto Rein,6832
7,Alison Danielson,2346.12,Alison Danielson,6832
8,Alison Dollar,964.52,Alison Dollar,6832
9,Alison Wakefield,1050.08,Alison Wakefield,6832


In [64]:
result.drop(columns=['name'], inplace=True)  # This will drop the column named Destination from the dataframe)

In [65]:
result.head(10)

,Employee,Actual Total Expenses,stipend
0,Aaron Salter,644.10,7377
1,Abigail Ratnofsky,2962.70,6086
2,Adam Kisthardt,1891.19,7377
3,Adam P. Jones,324.36,6832
4,Alan Butsch,2141.51,6913
5,Alberto Hook,1856.49,6913
6,Alberto Rein,330.08,6832
7,Alison Danielson,2346.12,6832
8,Alison Dollar,964.52,6832
9,Alison Wakefield,1050.08,6832


In [68]:
#This will compare the actual expense of each employee with the allocated stipend and 
#get employees that spent more than their approved stipend.
final_result = result[result['Actual Total Expenses'] > result['stipend']] 

In [69]:
final_result #This is the answer to Question 1

,Employee,Actual Total Expenses,stipend
49,Catherine Brewer,8208.76,6913
372,Thomas Jordan,13247.80,7377


## Question 2:
What location was most visited?Return the name of the City, State, and the total visits.

In [101]:
travels_dataset

,Employee,Travel Start Date,Travel End Date,Actual Total Expenses,Purpose Of Travel,State,City
0,Aaron Salter,2017-05-31,2017-06-02,644.10,Conference,MD,Ocean City
1,Abigail Ratnofsky,2016-01-24,2016-01-29,2962.70,Conference,CA,San Diego
2,Adam Kisthardt,2018-10-04,2018-10-09,1891.19,Conference,FL,Orlando
3,Adam P. Jones,2016-06-19,2016-06-21,324.36,Conference,MD,Ocean City
4,Alan Butsch,2019-11-19,2019-11-22,2141.51,Conference,CA,San Diego
...,...,...,...,...,...,...,...
413,Warren Jensen,2017-11-06,2017-11-09,502.45,Conference,MD,Ocean City
414,Warp Jensen,2019-03-25,2019-03-26,78.00,Conference,MD,Ocean City
415,Whitney Kujawa,2017-11-06,2017-11-09,567.00,Conference,MD,Ocean City
416,William Kinna,2017-05-21,2017-05-26,2163.67,Conference,NC,Charlotte


In [116]:
#This is to count the number of time each city was visted
result2 = travels_dataset.groupby(by=['City', 'State']).size().reset_index(name='TotalVisitsPerCity')
#result2 = travels_dataset.groupby(by=['City', 'Traveler']).size().reset_index(name='VisitCount')

In [111]:
#result2 = result2.sort_values(by='TotalVisitsPerCity') #This will sort in ascending by default, ascending=False for desc

In [170]:
# Find the index with the maximum value in 'TotalVisitsPerCity' column
max_index = result2['TotalVisitsPerCity'].idxmax()

# Get the row with the maximum value i.e series_with_max_value
question2_result = result2.loc[[max_index]]

In [171]:
question2_result

,City,State,TotalVisitsPerCity
13,Ocean City,MD,178


## Question 3:
How much was spent on travels by Troy Tippett? What state did he visit? What are the average household size and most common race found in that city?

In [134]:
travels_dataset

,Employee,Travel Start Date,Travel End Date,Actual Total Expenses,Purpose Of Travel,State,City
0,Aaron Salter,2017-05-31,2017-06-02,644.10,Conference,MD,Ocean City
1,Abigail Ratnofsky,2016-01-24,2016-01-29,2962.70,Conference,CA,San Diego
2,Adam Kisthardt,2018-10-04,2018-10-09,1891.19,Conference,FL,Orlando
3,Adam P. Jones,2016-06-19,2016-06-21,324.36,Conference,MD,Ocean City
4,Alan Butsch,2019-11-19,2019-11-22,2141.51,Conference,CA,San Diego
...,...,...,...,...,...,...,...
413,Warren Jensen,2017-11-06,2017-11-09,502.45,Conference,MD,Ocean City
414,Warp Jensen,2019-03-25,2019-03-26,78.00,Conference,MD,Ocean City
415,Whitney Kujawa,2017-11-06,2017-11-09,567.00,Conference,MD,Ocean City
416,William Kinna,2017-05-21,2017-05-26,2163.67,Conference,NC,Charlotte


In [135]:
#This is to select only entries where employee name is 'Troy Tippett'
data_quest3 = travels_dataset[travels_dataset['Employee'] == 'Troy Tippett']

In [136]:
#Bringing out only columns that are required for the analysis from the dataset
data_quest3 = data_quest3[['Employee', 'Actual Total Expenses', 'State', 'City']]

In [137]:
#Viewing the output of the selected entry/entries
data_quest3

,Employee,Actual Total Expenses,State,City
407,Troy Tippett,4616.69,FL,Orlando


In [138]:
#Importing the demography dataset using panda.read_csv()
demo_data = pd.read_csv('demography_data.csv')

In [154]:
#Extracting records of series where city is 'orlando'
demo_data3 = demo_data[demo_data['city'] == 'Orlando']

In [155]:
type(demo_data3)

pandas.core.frame.DataFrame

In [156]:
#merge
result = data_quest3.merge(demo_data3, how='inner', left_on='City', right_on='city')

In [162]:

result #To view the merged dataset of Orlando's demographic data with the employee Troy Tippett


,Employee,Actual Total Expenses,State,City,Unnamed: 0,city,state,median_age,male_population,female_population,total_population,number_of_veterans,foreign_born,average_household_size,state_code,race,count
0,Troy Tippett,4616.69,FL,Orlando,482,Orlando,Florida,33.1,130940.0,139977.0,270917,12782.0,50558.0,2.42,FL,Asian,11124
1,Troy Tippett,4616.69,FL,Orlando,665,Orlando,Florida,33.1,130940.0,139977.0,270917,12782.0,50558.0,2.42,FL,Black or African-American,68081
2,Troy Tippett,4616.69,FL,Orlando,1196,Orlando,Florida,33.1,130940.0,139977.0,270917,12782.0,50558.0,2.42,FL,American Indian and Alaska Native,2374
3,Troy Tippett,4616.69,FL,Orlando,2080,Orlando,Florida,33.1,130940.0,139977.0,270917,12782.0,50558.0,2.42,FL,Hispanic or Latino,89306
4,Troy Tippett,4616.69,FL,Orlando,2300,Orlando,Florida,33.1,130940.0,139977.0,270917,12782.0,50558.0,2.42,FL,White,179121


In [168]:
#To now sort the merged output to select the most common race found in that city
highest_count = result['count'].idxmax() #To find the index number of maximum count value

###Note: [[]] is required to get the output as dataframe, if a single [] is used it returns a dataseries instead###
final_result = result.loc[[highest_count]] # This will select the row that has the mx count value

In [169]:
final_result

,Employee,Actual Total Expenses,State,City,Unnamed: 0,city,state,median_age,male_population,female_population,total_population,number_of_veterans,foreign_born,average_household_size,state_code,race,count
4,Troy Tippett,4616.69,FL,Orlando,2300,Orlando,Florida,33.1,130940.0,139977.0,270917,12782.0,50558.0,2.42,FL,White,179121


In [174]:
final_result_question3 = final_result[['Employee', 'Actual Total Expenses', 'State', 'City', 'average_household_size', 'race']]

In [175]:
final_result_question3

,Employee,Actual Total Expenses,State,City,average_household_size,race
4,Troy Tippett,4616.69,FL,Orlando,2.42,White


## Question 4: 
How many of their employees visited Florida? What are the male and female population statistics?

In [244]:
#The travel_dataset
quest4 = pd.read_csv('cleaned_bissan_travels_data.csv')

In [245]:
quest4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             418 non-null    int64  
 1   Employee               418 non-null    object 
 2   Travel Start Date      418 non-null    object 
 3   Travel End Date        418 non-null    object 
 4   Actual Total Expenses  418 non-null    float64
 5   Purpose Of Travel      418 non-null    object 
 6   State                  418 non-null    object 
 7   City                   418 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 26.3+ KB


In [246]:
def remove_white_space(arg):
    if type(arg) == str:
        return arg.strip()
    return arg
    

In [262]:
#travels_dataset[travels_dataset['City'] == 'Orlando']
# travels_dataset['State'] = travels_dataset['State'].unique()
#travels_dataset['State'] = travels_dataset['State'].apply(remove_white_space)
quest4['State'] = quest4['State'].apply(remove_white_space)

In [263]:
quest4.head(10)

,Unnamed: 0,Employee,Travel Start Date,Travel End Date,Actual Total Expenses,Purpose Of Travel,State,City
0,0,Aaron Salter,2017-05-31,2017-06-02,644.10,Conference,MD,Ocean City
1,1,Abigail Ratnofsky,2016-01-24,2016-01-29,2962.70,Conference,CA,San Diego
2,2,Adam Kisthardt,2018-10-04,2018-10-09,1891.19,Conference,FL,Orlando
3,3,Adam P. Jones,2016-06-19,2016-06-21,324.36,Conference,MD,Ocean City
4,4,Alan Butsch,2019-11-19,2019-11-22,2141.51,Conference,CA,San Diego
5,5,Alberto Hook,2016-04-17,2016-04-22,1856.49,Conference,TX,Austin
6,6,Alberto Rein,2019-09-08,2019-09-12,330.08,Conference,MD,Ocean City
7,7,Alison Danielson,2019-09-23,2019-09-26,2346.12,Training,CA,Palm Springs
8,8,Alison Dollar,2015-06-17,2015-06-19,964.52,Conference,MD,Ocean City
9,9,Alison Wakefield,2015-06-28,2015-06-30,1050.08,Conference,MD,Ocean City


In [264]:
#
quest4_florida = quest4[quest4['State'] == 'FL']

In [265]:
quest4_florida.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56 entries, 2 to 407
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             56 non-null     int64  
 1   Employee               56 non-null     object 
 2   Travel Start Date      56 non-null     object 
 3   Travel End Date        56 non-null     object 
 4   Actual Total Expenses  56 non-null     float64
 5   Purpose Of Travel      56 non-null     object 
 6   State                  56 non-null     object 
 7   City                   56 non-null     object 
dtypes: float64(1), int64(1), object(6)
memory usage: 3.9+ KB


In [252]:
#unique_states = travels_dataset['State'].unique()
#print(unique_states)

## Note:
By using reset_index(name='Count'), you're creating a new DataFrame with two columns: 'State' and 'Count', where 'Count' represents the count of occurrences of each state in your original quest4_florida DataFrame

In [285]:
#To count the number of times employees visited Florida:
#Group the DataFrame by 'State' and count the occurrences
quest4_florida_count = quest4_florida.groupby(by=['State']).size().reset_index(name='Count')

In [286]:
quest4_florida_count

,State,Count
0,FL,56


In [287]:
#Merge this output of visit counts with the demography dataset
#Importing the demography dataset using panda.read_csv()
demo_data = pd.read_csv('demography_data.csv')

In [288]:
#Extracting records of series where city is 'orlando'
demo_data_quest4 = demo_data[demo_data['state_code'] == 'FL']

In [289]:
demo_data_quest4.tail(3)

,Unnamed: 0,city,state,median_age,male_population,female_population,total_population,number_of_veterans,foreign_born,average_household_size,state_code,race,count
2804,2804,Spring Hill,Florida,45.1,48295.0,54902.0,103197,9737.0,7893.0,2.58,FL,Hispanic or Latino,15838
2821,2821,Deltona,Florida,39.9,44853.0,43621.0,88474,5664.0,7098.0,2.89,FL,Black or African-American,12056
2852,2852,Weston,Florida,38.6,32956.0,36991.0,69947,1507.0,30876.0,3.34,FL,White,61021


In [306]:
# Calculate the sum of 'male_population' and 'female_population'

demo_data_quest4_aggregate = demo_data_quest4.agg({'male_population': ['sum'], 'female_population': ['sum']})
demo_data_quest4_aggregate = demo_data_quest4_aggregate.reset_index()

In [307]:
demo_data_quest4_aggregate

,index,male_population,female_population
0,sum,15461937.0,16626425.0


In [308]:
quest4_florida_count

,State,Count
0,FL,56


In [309]:
#Both dataframes don't have a common column, so I create a temporary common column ('key') with the same value for all
# rows in both DataFrames to perform the cross-join. The temporary column is then dropped after the join is completed.
result = quest4_florida_count.assign(key=1).merge(demo_data_quest4_aggregate.assign(key=1), on='key').drop('key', axis=1)


In [311]:
#Cleaned out the index column to make the result more tidy.
result = result.drop('index', axis=1)

In [312]:
#Visualizing the result for question 4
result

,State,Count,male_population,female_population
0,FL,56,15461937.0,16626425.0


## Qeustion 5:
Which state has the most veterans? Which employees travelled there?

In [402]:
#Importing the demography dataset using panda.read_csv()
quest5 = pd.read_csv('demography_data.csv')

In [411]:
quest5.drop_duplicates()

,Unnamed: 0,city,state,median_age,male_population,female_population,total_population,number_of_veterans,foreign_born,average_household_size,state_code,race,count
0,0,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402
1,1,Peoria,Illinois,33.1,56229.0,62432.0,118661,6634.0,7517.0,2.40,IL,American Indian and Alaska Native,1343
2,2,O'Fallon,Missouri,36.0,41762.0,43270.0,85032,5783.0,3269.0,2.77,MO,Hispanic or Latino,2583
3,3,Hampton,Virginia,35.5,66214.0,70240.0,136454,19638.0,6204.0,2.48,VA,Black or African-American,70303
4,4,Lakewood,Colorado,37.7,76013.0,76576.0,152589,9988.0,14169.0,2.29,CO,Hispanic or Latino,33630
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2886,2886,Tyler,Texas,33.9,50422.0,53283.0,103705,4813.0,8225.0,2.59,TX,Hispanic or Latino,21536
2887,2887,Mission Viejo,California,44.9,48849.0,48314.0,97163,4713.0,17308.0,2.85,CA,Black or African-American,2066
2888,2888,Compton,California,30.1,49264.0,49184.0,98448,897.0,30660.0,4.08,CA,Black or African-American,26395
2889,2889,Buffalo,New York,33.1,124537.0,133529.0,258066,11231.0,24630.0,2.27,NY,White,130078


In [412]:
result2 = quest5.sort_values(by='number_of_veterans') 

In [413]:
quest5['number_of_veterans'] = quest5['number_of_veterans'].fillna(0)

In [414]:
quest5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              2891 non-null   int64  
 1   city                    2891 non-null   object 
 2   state                   2891 non-null   object 
 3   median_age              2891 non-null   float64
 4   male_population         2888 non-null   float64
 5   female_population       2888 non-null   float64
 6   total_population        2891 non-null   int64  
 7   number_of_veterans      2891 non-null   float64
 8   foreign_born            2878 non-null   float64
 9   average_household_size  2875 non-null   float64
 10  state_code              2891 non-null   object 
 11  race                    2891 non-null   object 
 12  count                   2891 non-null   int64  
dtypes: float64(6), int64(3), object(4)
memory usage: 293.7+ KB


In [415]:
result2.tail(20)

,Unnamed: 0,city,state,median_age,male_population,female_population,total_population,number_of_veterans,foreign_born,average_household_size,state_code,race,count
2511,2511,Los Angeles,California,35.0,1958998.0,2012898.0,3971896,85417.0,1485425.0,2.86,CA,Asian,512999
1620,1620,Los Angeles,California,35.0,1958998.0,2012898.0,3971896,85417.0,1485425.0,2.86,CA,American Indian and Alaska Native,63758
680,680,Los Angeles,California,35.0,1958998.0,2012898.0,3971896,85417.0,1485425.0,2.86,CA,Hispanic or Latino,1936732
1334,1334,Los Angeles,California,35.0,1958998.0,2012898.0,3971896,85417.0,1485425.0,2.86,CA,Black or African-American,404868
2167,2167,Los Angeles,California,35.0,1958998.0,2012898.0,3971896,85417.0,1485425.0,2.86,CA,White,2177650
169,169,San Diego,California,34.5,693826.0,701081.0,1394907,92489.0,373842.0,2.73,CA,Black or African-American,111650
1407,1407,San Diego,California,34.5,693826.0,701081.0,1394907,92489.0,373842.0,2.73,CA,Hispanic or Latino,425414
1891,1891,San Diego,California,34.5,693826.0,701081.0,1394907,92489.0,373842.0,2.73,CA,Asian,267222
707,707,San Diego,California,34.5,693826.0,701081.0,1394907,92489.0,373842.0,2.73,CA,White,949388
1153,1153,San Diego,California,34.5,693826.0,701081.0,1394907,92489.0,373842.0,2.73,CA,American Indian and Alaska Native,16496


In [363]:
quest5_maxvet = quest5['number_of_veterans'].idxmax()

In [364]:
quest5_result = quest5.loc[[quest5_maxvet]]

In [365]:
quest5_result

,Unnamed: 0,city,state,median_age,male_population,female_population,total_population,number_of_veterans,foreign_born,average_household_size,state_code,race,count
796,796,New York,New York,36.0,4081698.0,4468707.0,8550405,156961.0,3212500.0,2.68,NY,American Indian and Alaska Native,90923


In [396]:
#Import the employee travel dataset
travel_data = pd.read_csv('cleaned_bissan_travels_data.csv')

In [397]:
checker = travel_data['State'] == 'NY'

In [398]:
#Added this step to verify it is actually an null output by looking thru the csv to confirm it
#checker = checker.to_csv('test.csv')


In [376]:
#Create a func to remove trailing whitespace from State values
def remove_white_space(arg):
    if type(arg) == str:
        return arg.strip()
    return arg


In [377]:
travel_data['State'] = travel_data['State'].apply(remove_white_space)

In [378]:
travel_data

,Unnamed: 0,Employee,Travel Start Date,Travel End Date,Actual Total Expenses,Purpose Of Travel,State,City
0,0,Aaron Salter,2017-05-31,2017-06-02,644.10,Conference,MD,Ocean City
1,1,Abigail Ratnofsky,2016-01-24,2016-01-29,2962.70,Conference,CA,San Diego
2,2,Adam Kisthardt,2018-10-04,2018-10-09,1891.19,Conference,FL,Orlando
3,3,Adam P. Jones,2016-06-19,2016-06-21,324.36,Conference,MD,Ocean City
4,4,Alan Butsch,2019-11-19,2019-11-22,2141.51,Conference,CA,San Diego
...,...,...,...,...,...,...,...,...
413,413,Warren Jensen,2017-11-06,2017-11-09,502.45,Conference,MD,Ocean City
414,414,Warp Jensen,2019-03-25,2019-03-26,78.00,Conference,MD,Ocean City
415,415,Whitney Kujawa,2017-11-06,2017-11-09,567.00,Conference,MD,Ocean City
416,416,William Kinna,2017-05-21,2017-05-26,2163.67,Conference,NC,Charlotte


In [416]:
travel_data_quest5 = travel_data[travel_data['State'] == 'NY']

In [417]:
travel_data_quest5['State'].unique()

array([], dtype=object)

In [418]:
travel_data_quest5

,Unnamed: 0,Employee,Travel Start Date,Travel End Date,Actual Total Expenses,Purpose Of Travel,State,City


In [ ]:
#Question 5 is empty as no employee travelled to NewYork from the Travel_set data.


## Question 6:
What is the gender breakdown of the most populated city?

In [421]:
#Importing the demography dataset using panda.read_csv()
quest6 = pd.read_csv('demography_data.csv')

In [423]:
quest6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              2891 non-null   int64  
 1   city                    2891 non-null   object 
 2   state                   2891 non-null   object 
 3   median_age              2891 non-null   float64
 4   male_population         2888 non-null   float64
 5   female_population       2888 non-null   float64
 6   total_population        2891 non-null   int64  
 7   number_of_veterans      2878 non-null   float64
 8   foreign_born            2878 non-null   float64
 9   average_household_size  2875 non-null   float64
 10  state_code              2891 non-null   object 
 11  race                    2891 non-null   object 
 12  count                   2891 non-null   int64  
dtypes: float64(6), int64(3), object(4)
memory usage: 293.7+ KB


In [424]:
quest6_maxvet = quest6['total_population'].idxmax()

In [425]:
quest6_result = quest6.loc[[quest6_maxvet]]

In [427]:
quest6_result = quest6_result[['city', 'male_population', 'female_population', 'total_population']]

In [428]:
final_result = print(quest6_result)

         city  male_population  female_population  total_population
796  New York        4081698.0          4468707.0           8550405


## Question 7:
How many veterans exist in the state visited by Evan Clarke?

In [454]:
#The travel_dataset
quest7 = pd.read_csv('cleaned_bissan_travels_data.csv')

In [455]:
quest7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             418 non-null    int64  
 1   Employee               418 non-null    object 
 2   Travel Start Date      418 non-null    object 
 3   Travel End Date        418 non-null    object 
 4   Actual Total Expenses  418 non-null    float64
 5   Purpose Of Travel      418 non-null    object 
 6   State                  418 non-null    object 
 7   City                   418 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 26.3+ KB


In [456]:
#This is to select only entries where employee name is 'Evan Clarke'
data_quest7 = quest7[quest7['Employee'] == 'Evan Clarke']

In [457]:
data_quest7

,Unnamed: 0,Employee,Travel Start Date,Travel End Date,Actual Total Expenses,Purpose Of Travel,State,City
120,120,Evan Clarke,2018-11-05,2018-11-08,595.42,Conference,MD,Ocean City


In [458]:
#Importing the demography dataset using panda.read_csv()
demo_data = pd.read_csv('demography_data.csv')

In [459]:
demo_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              2891 non-null   int64  
 1   city                    2891 non-null   object 
 2   state                   2891 non-null   object 
 3   median_age              2891 non-null   float64
 4   male_population         2888 non-null   float64
 5   female_population       2888 non-null   float64
 6   total_population        2891 non-null   int64  
 7   number_of_veterans      2878 non-null   float64
 8   foreign_born            2878 non-null   float64
 9   average_household_size  2875 non-null   float64
 10  state_code              2891 non-null   object 
 11  race                    2891 non-null   object 
 12  count                   2891 non-null   int64  
dtypes: float64(6), int64(3), object(4)
memory usage: 293.7+ KB


In [460]:
demo_data['number_of_veterans'] = demo_data['number_of_veterans'].fillna(0)

In [461]:
demo_data['city'].unique()

array(['Newark', 'Peoria', "O'Fallon", 'Hampton', 'Lakewood', 'Mesa',
       'Bryan', 'Garland', 'Springfield', 'Flint', 'Tacoma', 'Waterbury',
       'Eagan', 'Fort Smith', 'Carmichael', 'Daly City', 'Murrieta',
       'Brentwood', 'Quincy', 'Boulder', 'Evanston', 'Dothan', 'Antioch',
       'Madison', 'Victorville', 'Billings', 'Sandy Springs', 'Iowa City',
       'Largo', 'Shreveport', 'Kalamazoo', 'Round Rock', 'Richmond',
       'Columbia', 'Sandy', 'Hoover', 'Fullerton', 'Rapid City',
       'Newport Beach', 'Columbus', 'Palm Coast', 'Albany',
       'Jurupa Valley', 'Providence', 'Decatur', 'Scranton', 'Anaheim',
       'Rochester', 'Buffalo', 'Santa Ana', 'Sunnyvale',
       'Augusta-Richmond County consolidated government', 'Milwaukee',
       'Conroe', 'Savannah', 'German', 'Costa Mesa', 'New Haven',
       'Westland', 'Waldorf', 'Cedar Rapids', 'Troy', 'Kirkland',
       'Santa Barbara', 'Lake Forest', 'Chino', 'Tucson', 'Boise',
       'Joliet', 'Metairie', 'Moreno Valley',

In [462]:
#Create a func to remove trailing whitespace from State values
def remove_white_space(arg):
    if type(arg) == str:
        return arg.strip()
    return arg


In [466]:
demo_data['state_code'] = demo_data['state_code'].apply(remove_white_space)

In [467]:
demo_data['state_code'].unique()

array(['NJ', 'IL', 'MO', 'VA', 'CO', 'AZ', 'TX', 'MI', 'WA', 'CT', 'MN',
       'AR', 'CA', 'NY', 'MA', 'AL', 'WI', 'MT', 'GA', 'IA', 'FL', 'LA',
       'UT', 'SD', 'RI', 'PA', 'MD', 'ID', 'OK', 'NV', 'OH', 'NM', 'NC',
       'SC', 'PR', 'OR', 'AK', 'MS', 'ND', 'KS', 'DE', 'TN', 'ME', 'IN',
       'NE', 'KY', 'DC', 'NH', 'HI'], dtype=object)

In [468]:
maryland_data = demo_data[demo_data['state_code'] == 'MD']

In [469]:
maryland_data

,Unnamed: 0,city,state,median_age,male_population,female_population,total_population,number_of_veterans,foreign_born,average_household_size,state_code,race,count
59,59,German,Maryland,34.9,41115.0,43007.0,84122,2443.0,27877.0,2.95,MD,Black or African-American,22273
63,63,Waldorf,Maryland,33.6,35640.0,39872.0,75512,6932.0,5954.0,2.69,MD,White,26788
97,97,Frederick,Maryland,36.1,33146.0,36336.0,69482,3870.0,14211.0,2.48,MD,White,49181
147,147,Columbia,Maryland,37.9,52202.0,51265.0,103467,6526.0,23249.0,2.68,MD,White,58343
253,253,Frederick,Maryland,36.1,33146.0,36336.0,69482,3870.0,14211.0,2.48,MD,Asian,5711
300,300,Rockville,Maryland,38.1,31205.0,35793.0,66998,1990.0,25047.0,2.60,MD,Asian,17370
313,313,Ellicott City,Maryland,42.4,34406.0,36526.0,70932,3856.0,17587.0,2.73,MD,Hispanic or Latino,1602
369,369,Columbia,Maryland,37.9,52202.0,51265.0,103467,6526.0,23249.0,2.68,MD,American Indian and Alaska Native,488
390,390,Frederick,Maryland,36.1,33146.0,36336.0,69482,3870.0,14211.0,2.48,MD,American Indian and Alaska Native,1333
437,437,Baltimore,Maryland,34.7,294027.0,327822.0,621849,29540.0,49857.0,2.51,MD,White,207274


In [444]:
demo_data.head(10)

,Unnamed: 0,city,state,median_age,male_population,female_population,total_population,number_of_veterans,foreign_born,average_household_size,state_code,race,count
0,0,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402
1,1,Peoria,Illinois,33.1,56229.0,62432.0,118661,6634.0,7517.0,2.40,IL,American Indian and Alaska Native,1343
2,2,O'Fallon,Missouri,36.0,41762.0,43270.0,85032,5783.0,3269.0,2.77,MO,Hispanic or Latino,2583
3,3,Hampton,Virginia,35.5,66214.0,70240.0,136454,19638.0,6204.0,2.48,VA,Black or African-American,70303
4,4,Lakewood,Colorado,37.7,76013.0,76576.0,152589,9988.0,14169.0,2.29,CO,Hispanic or Latino,33630
5,5,Mesa,Arizona,36.9,234998.0,236835.0,471833,31808.0,57492.0,2.68,AZ,American Indian and Alaska Native,16044
6,6,Bryan,Texas,29.4,41761.0,40345.0,82106,3602.0,12014.0,2.55,TX,Black or African-American,11914
7,7,Garland,Texas,34.5,116406.0,120430.0,236836,10407.0,62975.0,3.12,TX,Asian,27217
8,8,Springfield,Illinois,38.8,55639.0,62170.0,117809,7525.0,4264.0,2.22,IL,Asian,3871
9,9,Flint,Michigan,35.3,48984.0,49313.0,98297,3757.0,2138.0,2.38,MI,Asian,657


In [438]:
demo_data = demo_data[demo_data['city'] == 'Ocean City']

In [439]:
demo_data

,Unnamed: 0,city,state,median_age,male_population,female_population,total_population,number_of_veterans,foreign_born,average_household_size,state_code,race,count


In [472]:
#There is to merge both dataframes to get the rows where city = Ocean City and State = MD
result_question7 = data_quest7.merge(demo_data, how='inner', left_on='City', right_on='city')

In [473]:
result_question7

,Unnamed: 0_x,Employee,Travel Start Date,Travel End Date,Actual Total Expenses,Purpose Of Travel,State,City,Unnamed: 0_y,city,...,median_age,male_population,female_population,total_population,number_of_veterans,foreign_born,average_household_size,state_code,race,count


In [ ]:
# This returned an empty dataset as there is no demographic data for the city employee 'Evan Clarke' visited.

## Question 8:
What month has the most travels?

In [524]:
#The travel_dataset
quest8 = pd.read_csv('cleaned_bissan_travels_data.csv')

In [525]:
quest8.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             418 non-null    int64  
 1   Employee               418 non-null    object 
 2   Travel Start Date      418 non-null    object 
 3   Travel End Date        418 non-null    object 
 4   Actual Total Expenses  418 non-null    float64
 5   Purpose Of Travel      418 non-null    object 
 6   State                  418 non-null    object 
 7   City                   418 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 26.3+ KB


In [526]:
#Repeating the process for Travel End Date
#First, apply the remove_white_space func should it exist in the data before converting to avoid error
#quest8['Travel Start Date'] = quest8['Travel Start Date'].apply(remove_white_space)
quest8['Travel Start Date'] = pd.to_datetime(quest8['Travel Start Date'])
quest8['Travel End Date'] = pd.to_datetime(quest8['Travel End Date'])

In [527]:
quest8.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Unnamed: 0             418 non-null    int64         
 1   Employee               418 non-null    object        
 2   Travel Start Date      418 non-null    datetime64[ns]
 3   Travel End Date        418 non-null    datetime64[ns]
 4   Actual Total Expenses  418 non-null    float64       
 5   Purpose Of Travel      418 non-null    object        
 6   State                  418 non-null    object        
 7   City                   418 non-null    object        
dtypes: datetime64[ns](2), float64(1), int64(1), object(4)
memory usage: 26.3+ KB


In [528]:
# Extract the month from the 'Travel Start Date' column
quest8['Travel Start Month'] = quest8['Travel Start Date'].dt.month
quest8['Travel End Month'] = quest8['Travel End Date'].dt.month

In [529]:
#quest8.drop('Month', axis=1)

In [530]:
final_result = quest8[quest8['Travel End Month'] > quest8['Travel Start Month']] 

In [531]:
final_result

,Unnamed: 0,Employee,Travel Start Date,Travel End Date,Actual Total Expenses,Purpose Of Travel,State,City,Travel Start Month,Travel End Month
0,0,Aaron Salter,2017-05-31,2017-06-02,644.10,Conference,MD,Ocean City,5,6
48,48,Casey Diaz,2019-03-31,2019-04-06,3450.07,Training,TX,Austin,3,4
51,51,Catherine Brewer,2018-10-28,2018-11-02,3283.52,Training,TX,Austin,10,11
69,69,Dan Berkman,2016-05-31,2016-06-02,503.23,Conference,MD,Ocean City,5,6
71,71,Dan Gunn,2019-09-29,2019-10-03,1525.76,Conference,CO,Aurora,9,10
82,82,Dave Anderson,2017-05-29,2017-06-02,1783.72,Conference,NV,Las Vegas,5,6
85,85,David Hinebaugh,2019-04-29,2019-05-03,152.50,Conference,CA,San Diego,4,5
99,99,Dimitry Ruvin,2017-11-26,2017-12-01,2195.32,Other,NV,Las Vegas,11,12
106,106,Earl Stoddard,2018-05-29,2018-06-01,303.27,Conference,MD,Ocean City,5,6
119,119,Eugene Curtis,2016-04-25,2016-05-06,5579.14,Training,FL,Orlando,4,5


In [532]:
# Group by 'Travel Start Month' and count the occurrences, naming the result column 'Count'
final_result8 = final_result.groupby(by=['Travel Start Month']).size().reset_index(name='Count')

In [533]:
final_result8

,Travel Start Month,Count
0,1,1
1,3,2
2,4,6
3,5,15
4,6,1
5,7,2
6,8,1
7,9,10
8,10,3
9,11,2


In [534]:
max_travel = final_result8['Count'].idxmax()
final_quest8 = final_result8.loc[max_travel]

In [521]:
#final_result8 = final_result8['Count'].max()

In [535]:
final_quest8

Travel Start Month     5
Count                 15
Name: 3, dtype: int64

In [ ]:
#The 5th month has the most travels = 15 